In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm

In [2]:

pid = [] # 제품 아이디
category = input()
for page in tqdm(range(int(input()))):
    url = f'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n=96&page={page}&req_ref=search&q={category}&stat_device=w&stat_category_required=1&version=4'
    response = requests.get(url)
    datas = response.json()['list']
    
    ids = [data['pid'] for data in datas] 
    pid.extend(ids)# 추가
items=[]
for i in pid: # 제품 아이디 하나씩 훑기
    url = f'https://api.bunjang.co.kr/api/1/product/{i}/detail_info.json?version=4'
    response = requests.get(url) # 리퀘스트
    try:
        details = response.json()['item_info'] #제이슨 파일을 그대로 가져오되 카테고리 이름, 페이옵션을 삭제
        details.pop('category_name')
        details.pop('pay_option')
        items.append(details)# 가져오면 딕셔너리 형태임
    except:
        print('error')

아이패드프로12.9형5세대
100


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.41it/s]


error


In [10]:
#데이터프레임화
df = pd.DataFrame(items)
bunjang_df = df[['name','price','location','description_for_detail','status','update_time','uid']]
bunjang_df = bunjang_df.rename({'name':'title','location':'region','description_for_detail':'desc','uid':'seller'},axis='columns')
#bunjang_df['link'] = 'https://m.bunjang.co.kr/products/'+ bunjang_df['pid']
#bunjang_df['market'] = '번개장터' -> 딱히 필요없을듯
#bunjang_df['keyword'] = category -> 키워드가 표시되게 하고싶으면 추가
#bunjang_df.drop(['pid'], axis=1)

# 전처리
bunjang_df.reset_index(drop = True)
bunjang_df['desc'] = bunjang_df['desc'] \
.replace("'", '') \
.replace(r'\s+', ' ', regex=True) \
.str.strip() \
.str[:255]
bunjang_df = bunjang_df[bunjang_df['desc'].str.strip().astype(bool)]

In [11]:
from datetime import datetime
# 일-월-년 으로 출력할려면 사용

bunjang_df['update_time']=bunjang_df['update_time'].map(lambda x : datetime.fromtimestamp(x).strftime('%y-%m-%d') ) 
bunjang_df['update_time']

0      22-07-07
1      22-07-07
2      22-07-07
3      22-07-06
4      22-07-06
         ...   
154    22-04-08
155    22-04-08
156    22-04-06
157    22-04-01
158    21-11-30
Name: update_time, Length: 159, dtype: object

In [12]:
#bunjang_df.query('desc.str.contains("매입|삽니다|구매|최고가|전기종")', engine='python')

drop_list = bunjang_df.query('desc.str.contains("매입|삽니다|구매|최고가|전기종")', engine='python').index
drop_bunjang_df = bunjang_df.drop(drop_list)


drop_bunjang_df.price = drop_bunjang_df.price.astype('float')

q1= drop_bunjang_df['price'].quantile(0.25)
q3= drop_bunjang_df['price'].quantile(0.75)
iqr=q3-q1
condition=drop_bunjang_df['price']>q3+1.5*iqr

drop_price_list = drop_bunjang_df[condition].index

drop_bunjang_df.drop(drop_price_list, inplace= True)

drop_bunjang_df.reset_index(inplace= True)

drop_bunjang_df.drop('index', axis= 1, inplace= True)

drop_bunjang_df

,title,price,region,desc,status,update_time,seller
0,아이패드 프로 5세대 M1 12.9형 256G미개봉,1120000.0,경기도 안성시 금광면,"미개봉 아이패드 프로12.9형 5세대 wifi버전 256G 스페이스 그레이,실버 가...",0,22-07-07,76258386
1,"아이패드 프로 5세대 M1 12.9형 스페이스그레이,실버 128G미개봉",1040000.0,경기도 안성시 금광면,M1 12.9형 아이패드 프로 128G 스페이스그레이 미개봉입니다 수량 스페이스그레...,0,22-07-07,76258386
2,(가격내림)아이패드프로5세대 12.9형 팝니다,1300000.0,인천광역시 남동구 논현1동,스페이스그레이모델에 올초에 구입해서 거의사용안한상품이고 보증기한도 아직 남아잇는상품...,0,22-07-07,12726998
3,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 256GB 그레이,1350000.0,서울특별시 영등포구 여의동,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 256GB 스페이스그레이 [ ...,0,22-07-06,77105922
4,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 128GB 실버,1240000.0,서울특별시 영등포구 여의동,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 128GB 실버 [ MHNG3...,0,22-07-06,77105922
...,...,...,...,...,...,...,...
103,아이패드 프로 5세대 12.9형 256g 셀룰러 급처,1300000.0,서울특별시 강북구 미아동,아이패드 프로 5세대 12.9형 256g 와이파이+셀룰러 스페이스 그레이 판매 합니...,3,22-04-11,769911
104,아이패드 프로 12.9형 5세대 와이파이 스페이스그레이,1200000.0,인천광역시 서구 청라3동,미개봉 새상품 입니다,3,22-04-08,75744914
105,"아이패드 프로 5세대 M1 12.9형 스페이스그레이,실버 128G미개봉",1040000.0,경기도 안성시 금광면,M1 12.9형 아이패드 프로 128G 스페이스그레이 미개봉입니다 수량 스페이스그레...,3,22-04-08,76258386
106,(미개봉)아이패드프로5세대 12.9형 256G 셀룰러,1390000.0,경상남도 창원시 성산구 반송동,제조연월 2022/03 스페이스 그레이 미개봉 새제품 입니다 ※ 안전결제/직거래/택...,3,22-04-01,74951464


In [15]:
drop_bunjang_df.drop_duplicates(['title','desc','price','seller'],inplace=True)
drop_bunjang_df.to_csv(f"번개장터크롤링_{category}.csv" ,encoding="utf-8")
drop_bunjang_df.head()

,title,price,region,desc,status,update_time,seller
0,아이패드 프로 5세대 M1 12.9형 256G미개봉,1120000.0,경기도 안성시 금광면,"미개봉 아이패드 프로12.9형 5세대 wifi버전 256G 스페이스 그레이,실버 가...",0,22-07-07,76258386
1,"아이패드 프로 5세대 M1 12.9형 스페이스그레이,실버 128G미개봉",1040000.0,경기도 안성시 금광면,M1 12.9형 아이패드 프로 128G 스페이스그레이 미개봉입니다 수량 스페이스그레...,0,22-07-07,76258386
2,(가격내림)아이패드프로5세대 12.9형 팝니다,1300000.0,인천광역시 남동구 논현1동,스페이스그레이모델에 올초에 구입해서 거의사용안한상품이고 보증기한도 아직 남아잇는상품...,0,22-07-07,12726998
3,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 256GB 그레이,1350000.0,서울특별시 영등포구 여의동,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 256GB 스페이스그레이 [ ...,0,22-07-06,77105922
4,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 128GB 실버,1240000.0,서울특별시 영등포구 여의동,애플 아이패드 프로 12.9형 5세대 M1 Wi-Fi 128GB 실버 [ MHNG3...,0,22-07-06,77105922
